In [207]:
import vk
import pandas as pd
import json
import time


In [208]:
v='5.89'
api = vk.API(access_token='vk1.a._KsTsa8wau2jsv2iCt80iIj-z2gu7Y978RoBwh4jnZOZymVH_nR4EFJ-7Jj0tqL1SAA6T9jnVOXKMLlGUL1O3-gH8KmBirDjBe7AbrzfBsd5voGelouCQTWcasNqpRpxUKjEIftzpbahGTPyIi5Coxu2lR_rtjIMMwgyyPpq3S_JlTpnKFcNTf5ClHu1hHySNtmEEEVTnWtVjhVsJCVCpQ',v=v)




In [222]:
users=['https://vk.com/yourdevils18','https://vk.com/privet.andreii','https://vk.com/danbase']
for i,user in enumerate(users):
    users[i]=user.replace('https://vk.com/','')


def getUserId(users):
    users_ids=[]
    for user in users:
        user_id=api.users.get(user_id=user)[0]['id']
        users_ids.append(user_id)
    return users_ids


users_ids=getUserId(users)
users_ids



[436090772, 371599152, 259775630]

In [230]:
df=pd.DataFrame()
df_main=pd.DataFrame()
df_inf=pd.DataFrame()
data_info=pd.DataFrame()

#Получение групп
def getGroups(user_id):
    time.sleep(1)
    data_gr=api.groups.get(user_id=user_id,count=1000)
    df[['Groups_count','Groups_id']] = pd.json_normalize(data_gr)


#Получение друзей
def getFriends(user_id):
    time.sleep(1)
    data_fr=api.friends.get(user_id=user_id,count=1000)
    df[['Friends_count','friends_id']] = pd.json_normalize(data_fr)

#Получение подписчиков
def getFollowers(user_id):
    time.sleep(1)
    data_fol=api.users.getFollowers(user_id=user_id,count=1000)
    df[['User_Folowers','user_f_ids']] = pd.json_normalize(data_fol)

#Получение информации со стены, включая текст
def getWall(owner_id):
    time.sleep(1)
    data_wall=api.wall.get(owner_id=owner_id,count=1000)
    df[['Wall_Q','Wall_count']]= pd.json_normalize(data_wall)

    time.sleep(1)
    data_wall_owner=api.wall.get(owner_id=owner_id,count=1000,filter='owner')
    df[['Wall_Q_owner','Wall_count_owner']]= pd.json_normalize(data_wall_owner)

    time.sleep(1)
    data_wall_owner=api.wall.get(owner_id=owner_id,count=1000,filter='others')
    df[['Wall_Q_others','Wall_count_others']]= pd.json_normalize(data_wall_owner)

    posts_strings=[]
    time.sleep(1)
    posts = api.wall.get(owner_id=owner_id,count=1000)['items']
    posts_strings.append([post['text'] for post in posts])
    df['posts_text']=posts_strings



#Получение количества фото с комментариями под ними
def getPhotos(owner_id):
    time.sleep(1)
    data_photos_com=api.photos.getAllComments(owner_id=owner_id,count=100)['count']
    df[['Photos_C']]= data_photos_com

    comments_strings = []
    time.sleep(1)
    comments = api.photos.getAllComments(owner_id=owner_id,count=100)['items']
    comments_strings.append([comment['text'] for comment in comments])
    df['comments_photo']=comments_strings

#Получение статуса профиля
def getStatus(user_id):
    time.sleep(1)
    data_status=api.status.get(user_id=user_id)
    df[['Status_txt']]= pd.json_normalize(data_status)

#Получение статуса аватарки профиля
def getAvatar(user_id):
    time.sleep(1)
    data_Avatar=api.users.get(user_ids=user_id,fields='has_photo')[0]['has_photo']
    df['Avatar']=data_Avatar

#Надо получить все что в fields(Пока только about получается)
def getUserInfo(user_id):
    time.sleep(1)
    data_info=api.users.get(user_ids=user_id,fields=['activities','about','sex','screen_name','interests','movies','books','bdate','music','relation'])
    data_info=pd.json_normalize(data_info,errors='ignore')
    return data_info

start_time = time.time()
for user_id in users_ids:
    print(user_id)
    getFriends(user_id)
    getFollowers(user_id)
    getGroups(user_id)
    getWall(user_id)
    getPhotos(user_id)
    getStatus(user_id)
    getAvatar(user_id)
    data_info = getUserInfo(user_id)
    df['user_id']=str(user_id)
    df_inf=pd.concat([df_inf,data_info],ignore_index=True)
    df_main=pd.concat([df_main,df],ignore_index=True)

df_inf=df_inf.drop(['relation_partner.id','relation_partner.first_name','relation_partner.last_name'], axis=1)

print("Success.\n Result saved in 'links.txt' file.\n Running time:", (time.time() - start_time) / 60, "minutes")

   Friends_count                                         friends_id  \
0             68  [9309774, 11097036, 14527278, 31696367, 509424...   

   User_Folowers                                         user_f_ids  \
0            150  [336442933, 213375153, 300740526, 516432917, 4...   

   Groups_count                                          Groups_id  Wall_Q  \
0           100  [187289777, 178282306, 73408049, 134506306, 15...       3   

                                          Wall_count  Wall_Q_owner  \
0  [{'is_pinned': 1, 'comments': {'can_post': 1, ...             3   

                                    Wall_count_owner  Wall_Q_others  \
0  [{'is_pinned': 1, 'comments': {'can_post': 1, ...              0   

  Wall_count_others posts_text  Photos_C  \
0                []   [💍❤, , ]        31   

                                      comments_photo Status_txt  Avatar  \
0  [Ну просто бомба😍🔥, Красотка ❤❤❤, [id392535378...                  1   

     user_id  
0  436090772  
   

In [231]:
df_main

,Friends_count,friends_id,User_Folowers,user_f_ids,Groups_count,Groups_id,Wall_Q,Wall_count,Wall_Q_owner,Wall_count_owner,Wall_Q_others,Wall_count_others,posts_text,Photos_C,comments_photo,Status_txt,Avatar,user_id
0,68,"[9309774, 11097036, 14527278, 31696367, 509424...",150,"[336442933, 213375153, 300740526, 516432917, 4...",100,"[187289777, 178282306, 73408049, 134506306, 15...",3,"[{'is_pinned': 1, 'comments': {'can_post': 1, ...",3,"[{'is_pinned': 1, 'comments': {'can_post': 1, ...",0,[],"[💍❤, , ]",31,"[Ну просто бомба😍🔥, Красотка ❤❤❤, [id392535378...",,1,436090772
1,220,"[21657, 557323, 835507, 18302249, 19736323, 23...",326,"[399592987, 48038351, 351674064, 379865667, 53...",47,"[28905875, 61287706, 66640066, 159146575, 8464...",0,[],0,[],0,[],[],7,"[, , [id196157427|Ян], в огороде работаю 💪🏻, Ч...",,1,371599152
2,144,"[3626517, 5115969, 8033142, 12008505, 14070034...",140,"[164782374, 226026450, 172275578, 225828677, 2...",7,"[29346, 97882810, 58120520, 171262993, 1435990...",0,[],0,[],0,[],[],0,[],,0,259775630


In [215]:
df_inf

,id,bdate,interests,books,about,movies,activities,music,relation,sex,screen_name,first_name,last_name,can_access_closed,is_closed
0,436090772,18.10,,,,,,,4.0,1,yourdevils18,Вероника,Аржановская,True,False
1,371599152,6.10,,,,,,,0.0,2,privet.andreii,Андрей,Раховецкий,True,False
2,259775630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,danbase,Даниил,Базанов,True,True


In [240]:
def get_latest_posts_from_groups(group_ids, number_of_posts, api_ver):
    all_posts = []
    req_counter = 0
    for group in group_ids:
        req_counter += 1
        if req_counter == 3:
            # VK API allows to make only 3 requests per second
            time.sleep(1)
            req_counter = 0
        latest_posts = api.wall.get(owner_id="-" + str(group), count=str(number_of_posts), filter="all", v=str(api_ver))
        latest_posts = latest_posts['items']
        latest_posts_ids = [[item['id'], item['owner_id'], item['date']] for item in latest_posts]
        all_posts.extend(latest_posts_ids)
    return all_posts


def get_latest_posts_from_friend(friend_ids, number_of_posts, api_ver):
    all_posts = []
    req_counter = 0
    for friend in friend_ids:
        req_counter += 1
        if req_counter == 3:
            # VK API allows to make only 3 requests per second
            time.sleep(1)
            req_counter = 0
        try:
            latest_posts = api.wall.get(owner_id=str(friend), count=str(number_of_posts), filter="all", v=str(api_ver))
        except Exception:
            # friend profile can be private/deleted
            continue
        latest_posts = latest_posts['items']
        latest_posts_ids = [[item['id'], item['owner_id'], item['date']] for item in latest_posts]
        all_posts.extend(latest_posts_ids)
    return all_posts


def filter_liked_group_posts(group_posts, user_id, days, api_ver):
    liked_group_posts = []
    req_counter = 0
    for post in group_posts:
        if (time.time() - post[2]) / (24 * 60 * 60) > days:
            continue
        req_counter += 1
        if req_counter == 3:
            time.sleep(1)
            req_counter = 0
        info = api.likes.isLiked(user_id=user_id, type="post", owner_id=str(post[1]), item_id=str(post[0]), v=str(api_ver))
        if info['liked'] == 1:
            liked_group_posts.append("https://vk.com/wall" + str(post[1]) + "_" + str(post[0]))
    return liked_group_posts


def filter_liked_friend_posts(friend_posts, user_id, days, api_ver):
    liked_friend_posts = []
    req_counter = 0
    for post in friend_posts:
        if (time.time() - post[2]) / (24 * 60 * 60) > days:
            continue
        req_counter += 1
        if req_counter == 3:
            time.sleep(1)
            req_counter = 0
        info = api.likes.isLiked(user_id=user_id, type="post", owner_id=str(post[1]), item_id=str(post[0]), v=str(api_ver))
        if info['liked'] == 1:
            liked_friend_posts.append("https://vk.com/wall" + str(post[1]) + "_" + str(post[0]))
    return liked_friend_posts


def user_liked_group_posts(user_id, number_of_posts, days, api_ver):
    subscriptions = api.users.getSubscriptions(user_id=str(user_id), v=str(api_ver))
    group_ids = subscriptions['groups']['items']
    all_posts = get_latest_posts_from_groups(group_ids, str(number_of_posts), str(api_ver))
    return filter_liked_group_posts(all_posts, str(user_id), days, str(api_ver))


def user_liked_friend_posts(user_id, number_of_posts, days, api_ver):
    friends = api.friends.get(user_id=str(user_id), v=str(api_ver))
    friend_ids = friends['items']
    all_posts = get_latest_posts_from_friend(friend_ids, str(number_of_posts), str(api_ver))
    return filter_liked_friend_posts(all_posts, str(user_id), days, str(api_ver))


def save_links_in_txt(links):
    output = open('links.txt', 'w')
    for link in links:
        print(link, file=output)
    output.close()


def main():
    df_likes_main=pd.DataFrame()
    for user_id in users_ids:
        number_of_posts = 100
        number_of_days = 31
        print("Processing...")
        print(user_id)

        start_time = time.time()

        #df_likes_gr=pd.DataFrame([user_id,user_liked_group_posts(user_id, number_of_posts, number_of_days, v)],columns=['user_id','links_gr'])

        df_likes_fr=pd.DataFrame([[user_id,user_liked_friend_posts(user_id, number_of_posts, number_of_days, v)]],columns=['user_id','links_fr'])

        df_likes_main=pd.concat([df_likes_main,df_likes_fr])

    print("Success.\n Result saved in 'links.txt' file.\n Running time:", (time.time() - start_time) / 60, "minutes")
    print(df_likes_main)


main()

Processing...
436090772
['https://vk.com/wall260436059_3988', 'https://vk.com/wall362624128_540']
Processing...
371599152
['https://vk.com/wall153397723_13771', 'https://vk.com/wall166393600_3500', 'https://vk.com/wall187216093_1313', 'https://vk.com/wall192252787_1022', 'https://vk.com/wall194740253_1643', 'https://vk.com/wall230940352_1824', 'https://vk.com/wall232887839_614', 'https://vk.com/wall253520847_2842', 'https://vk.com/wall259605432_1107', 'https://vk.com/wall267272297_135', 'https://vk.com/wall299277857_467', 'https://vk.com/wall303556910_2601', 'https://vk.com/wall516974744_2774', 'https://vk.com/wall519552199_539']
Processing...
259775630
['https://vk.com/wall175891758_1009']
Success.
 Result saved in 'links.txt' file.
 Running time: 1.4719680229822794 minutes
     user_id                                           links_fr
0  436090772  [https://vk.com/wall260436059_3988, https://vk...
0  371599152  [https://vk.com/wall153397723_13771, https://v...
0  259775630          